# 2.3 텐서 연산

- 첫 예제에서의 Dense layer를 간단히 분석해보면 다음과 같다.


- keras.layers.Dense(512, activation='relu')
    - 입력으로 2D 텐서를 받아서 출력으로 2D 텐서를 반환하는 함수
    - 더 자세히 풀어서 보면 다음과 같다.
    
    
- output = relu(dot(W, input) + b)
    - input(입력 2D 텐서), W(가중치 2D 텐서), bias(벡터)
    - 총 3가지 연산을 함
        1. input과 W의 dot product
        2. 1의 결과(2D 텐서)와 b의 덧셈
        3. ReLU 연산
    
    
- ReLU 연산
    - max(x, 0)
    - 입력이 0보다 크면 그대로 반환
    - 0보다 작으면 0을 반환
    
    
***Sequential 클래스의 add() method에 Dense 클래스가 추가될 때, Dense 객체의 build() 메서드가 호출되며 가중치(kernel) W와 편향(bias) b가 생성되고, Dense 객체의 kernel과 bias 인스턴스 변수에 저장됨***

## 2.3.1 원소별 연산 (element-wise operation)

- relu()와 덧셈은 원소별 연산
- 원소별 연산이란 텐서내의 원소끼리 이루어지는 연산을 뜻함
- 넘파이에 원소별 연산이 이미 구현되어 있고 훨씬 빠름

In [1]:
import numpy as np

# 직접 구현한 relu()
def naive_relu(x):
    assert len(x.shape) == 2
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
            
    return x


# 직접 구현한 add()
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
            
    return x


a = np.random.randint(-10, 10, (3, 3))
b = np.random.randint(-10, 10, (3, 3))

print("a")
print(a)
print("b")
print(b)
print("")

print("relu(a)")
print(naive_relu(a))
print("")

print("add(a, b)")
print(naive_add(a, b))
print("")


# numpy의 구현된 maximum(relu()와 동일한 기능)
print("numpy의 relu")
print(np.maximum(a, 0.))
print("")

# numpy의 구현된 add()
print("numpy의 add")
print(a + b)

a
[[-3  9 -9]
 [-7 -9  6]
 [-2  4  2]]
b
[[ -3  -7 -10]
 [ -5   8   3]
 [  9   3  -8]]

relu(a)
[[0 9 0]
 [0 0 6]
 [0 4 2]]

add(a, b)
[[ -6   2 -19]
 [-12  -1   9]
 [  7   7  -6]]

numpy의 relu
[[0. 9. 0.]
 [0. 0. 6.]
 [0. 4. 2.]]

numpy의 add
[[ -6   2 -19]
 [-12  -1   9]
 [  7   7  -6]]


## 2.3.2 브로드캐스팅
- 작은 텐서가 큰 텐서의 크기에 맞춰지는 것을 브로드캐스팅이라 함
- 브로드캐스팅의 2단계
    1. 큰 텐서의 ndim에 맞도록 작은 텐서에 축(브로드캐스팅 축이라 함)이 추가됨
    2. 작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복됨

In [48]:
# 직접 구현한 broadcasting
# 실제로는 알고리즘 수준에서 더 빠르게 구현되어 있을 것임, 간단히 구현만 해본 것
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
            
    return x


# 마지막 차원 맞춰주기 (5를 같게함)
a = np.random.randint(-10, 10, [10, 5])
b = np.random.randint(-10, 10, [5])
c = np.random.randint(10, size=[64, 3, 32, 10])
d = np.random.randint(10, size=[32, 10])

print("a :", a.shape)
print("b :", b.shape)
print("")

print("a+b (broadcasting) :", (a+b).shape)
print("naive_add_matrix_and_vector(a, b) :", naive_add_matrix_and_vector(a, b).shape)
print("")

# 이렇게도 broadcasting이 일어남
print("c :", c.shape)
print("d :", d.shape)
print("c+d (broadcasting) :", (c+d).shape)
print("")

a : (10, 5)
b : (5,)

a+b (broadcasting) : (10, 5)
naive_add_matrix_and_vector(a, b) : (10, 5)

c : (64, 3, 32, 10)
d : (32, 10)
c+d (broadcasting) : (64, 3, 32, 10)

